In [1]:
import pathlib
import warnings
from typing import List, Tuple

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm
from sklearn.decomposition import PCA
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import (
    explained_variance_score,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.multioutput import MultiOutputRegressor

In [2]:
def shuffle_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Shuffle the data in the DataFrame.
    """
    df_shuffled = df.copy()
    for col in df_shuffled.columns:
        if "metadata" in col.lower():
            continue  # skip metadata columns
        # permute the columns
        df_shuffled[col] = np.random.permutation(df_shuffled[col])
    return df_shuffled

In [3]:
# read in the data
sc_file_path = pathlib.Path("../results/train_sc_profile.parquet").resolve(strict=True)
sc_endpoint_file_path = pathlib.Path(
    "../results/train_sc_profile_terminal_time.parquet"
).resolve(strict=True)

model_dir = pathlib.Path("../models").resolve()
model_dir.mkdir(parents=True, exist_ok=True)
results_dir = pathlib.Path("../results").resolve()
results_dir.mkdir(parents=True, exist_ok=True)
sc_profile = pd.read_parquet(sc_file_path)
sc_endpoint_profile = pd.read_parquet(sc_endpoint_file_path)

In [4]:
# training_df_X =
training_df_X = sc_profile.loc[
    sc_profile["Metadata_Time"] == sc_profile["Metadata_Time"].max()
]

In [5]:
training_df_y = sc_endpoint_profile.loc[
    sc_endpoint_profile["Metadata_sc_unique_track_id"].isin(
        training_df_X["Metadata_sc_unique_track_id"]
    )
]
print(
    f"training_df_X shape: {training_df_X.shape}",
    training_df_X["Metadata_sc_unique_track_id"].nunique(),
)
print(
    f"training_df_y shape: {training_df_y.shape}",
    training_df_y["Metadata_sc_unique_track_id"].nunique(),
)
assert (
    training_df_X["Metadata_sc_unique_track_id"].nunique()
    == training_df_y["Metadata_sc_unique_track_id"].nunique()
)
assert (
    training_df_X["Metadata_sc_unique_track_id"].shape[0]
    == training_df_y["Metadata_sc_unique_track_id"].shape[0]
)
training_df_X_shuffled = training_df_X.copy()
training_df_X_shuffled = shuffle_data(training_df_X_shuffled)

training_df_X shape: (2300, 2379) 2300
training_df_y shape: (2300, 541) 2300


In [6]:
train_x_metadata = [x for x in training_df_X.columns if "Metadata" in x]
train_y_metadata = [y for y in training_df_y.columns if "Metadata" in y]
training_X_features = [x for x in training_df_X.columns if x not in train_x_metadata]
training_y_features = [y for y in training_df_y.columns if y not in train_y_metadata]

train_x_shuffled_metadata = [
    x for x in training_df_X_shuffled.columns if "Metadata" in x
]
train_y_shuffled_metadata = [y for y in training_df_y.columns if "Metadata" in y]
train_x_shuffled_features = [
    x for x in training_df_X_shuffled.columns if x not in train_x_shuffled_metadata
]

train_df_x_metadata = training_df_X[train_x_metadata]
train_df_y_metadata = training_df_y[train_y_metadata]
train_df_x_features = training_df_X[training_X_features]
train_df_y_features = training_df_y[training_y_features]
train_df_x_shuffled_metadata = training_df_X_shuffled[train_x_shuffled_metadata]
train_df_x_shuffled_features = training_df_X_shuffled[train_x_shuffled_features]

In [7]:
annexin_feature = "Cytoplasm_Intensity_IntegratedIntensity_AnnexinV"

In [8]:
# Define cross-validation strategy
cv = KFold(n_splits=5, shuffle=True, random_state=0)  # 5-fold cross-validation
# elastic net parameters
elastic_net_params = {
    "alpha": [0.1, 1.0, 10.0, 100.0, 1000.0],  # Regularization strength
    "l1_ratio": [0.1, 0.25, 0.5, 0.75, 1.0],  # l1_ratio = 1.0 is Lasso
    "max_iter": 10000,  # Increase max_iter for convergence
}
elastic_net_all_annexinv_features_model = MultiOutputRegressor(
    ElasticNetCV(
        alphas=elastic_net_params["alpha"],
        l1_ratio=elastic_net_params["l1_ratio"],
        cv=cv,
        random_state=0,
        max_iter=elastic_net_params["max_iter"],
    )
)
elastic_net_all_annexinv_features_model_shuffled = (
    elastic_net_all_annexinv_features_model
)
elastic_net_single_terminal_features_model = ElasticNetCV(
    alphas=elastic_net_params["alpha"],
    l1_ratio=elastic_net_params["l1_ratio"],
    cv=cv,
    random_state=0,
    max_iter=elastic_net_params["max_iter"],
)
elastic_net_single_terminal_features_model_shuffled = (
    elastic_net_single_terminal_features_model
)

In [9]:
dict_of_train_tests = {
    "single_feature": {
        "train": {
            "X": train_df_x_features,
            "y": train_df_y_features[annexin_feature],
            "x_metadata": train_df_x_metadata,
            "y_metadata": train_df_y_metadata,
            "model": elastic_net_single_terminal_features_model,
            "model_name": "elastic_net_single_terminal_features_model",
        },
        "train_shuffled": {
            "X": train_df_x_shuffled_features,
            "y": train_df_y_features[annexin_feature],
            "x_metadata": train_df_x_shuffled_metadata,
            "y_metadata": train_df_y_metadata,
            "model": elastic_net_single_terminal_features_model_shuffled,
            "model_name": "elastic_net_single_terminal_features_model_shuffled",
        },
    },
    "annexinV_features": {
        "train": {
            "X": train_df_x_features,
            "y": train_df_y_features,
            "x_metadata": train_df_x_metadata,
            "y_metadata": train_df_y_metadata,
            "model": elastic_net_all_annexinv_features_model,
            "model_name": "elastic_net_all_annexinv_features_model",
        },
        "train_shuffled": {
            "X": train_df_x_shuffled_features,
            "y": train_df_y_features,
            "x_metadata": train_df_x_shuffled_metadata,
            "y_metadata": train_df_y_metadata,
            "model": elastic_net_all_annexinv_features_model_shuffled,
            "model_name": "elastic_net_all_annexinv_features_model_shuffled",
        },
    },
}

In [10]:
# train the model
for model_type in dict_of_train_tests.keys():
    for train_test_key, train_test_data in tqdm.tqdm(
        dict_of_train_tests[model_type].items()
    ):
        if "test" in train_test_key:
            print(f"Skipping {train_test_key} as it is a test set.")
            continue
        print(f"Training model for {train_test_key}...{model_type}")
        X = train_test_data["X"]
        y = train_test_data["y"]
        x_metadata = dict_of_train_tests[model_type][train_test_key]["x_metadata"]
        y_metadata = dict_of_train_tests[model_type][train_test_key]["y_metadata"]
        print(
            f"X shape: {X.shape}, y shape: {y.shape}, x_metadata shape: {x_metadata.shape}, y_metadata shape: {y_metadata.shape}"
        )
        # find the number of NaNs
        num_nans_X = X.isna().sum().sum()
        num_nans_y = y.isna().sum().sum()
        print(f"Number of NaNs in X: {num_nans_X}, Number of NaNs in y: {num_nans_y}")
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=ConvergenceWarning)
            dict_of_train_tests[model_type][train_test_key]["model"].fit(X, y)

        # save the model
        model_path = (
            model_dir / f"{train_test_key}_{train_test_data['model_name']}.joblib"
        )
        joblib.dump(train_test_data["model"], model_path)
        dict_of_train_tests[model_type][train_test_key]["model_path"] = model_path

  0%|          | 0/2 [00:00<?, ?it/s]

Training model for train...single_feature
X shape: (2300, 2336), y shape: (2300,), x_metadata shape: (2300, 43), y_metadata shape: (2300, 31)
Number of NaNs in X: 0, Number of NaNs in y: 0


 50%|█████     | 1/2 [00:26<00:26, 26.16s/it]

Training model for train_shuffled...single_feature
X shape: (2300, 2336), y shape: (2300,), x_metadata shape: (2300, 43), y_metadata shape: (2300, 31)
Number of NaNs in X: 0, Number of NaNs in y: 0


  0%|          | 0/2 [00:00<?, ?it/s]

Training model for train...annexinV_features
X shape: (2300, 2336), y shape: (2300, 510), x_metadata shape: (2300, 43), y_metadata shape: (2300, 31)
Number of NaNs in X: 0, Number of NaNs in y: 0


  0%|          | 0/2 [2:00:31<?, ?it/s]


KeyboardInterrupt: 

training took approximately 637 minutes (10.6 hours) on a machine with 1 NVIDIA 3090 TI GPU.